In [1]:
import numpy as np
import paddle
import paddle.fluid as fluid
import random
import gym
from collections import deque

In [2]:
num_episodes = 500
num_exploration_episodes = 100
max_len_episode = 1000
batch_size = 32
learning_rate = 1e-3
gamma = 1.
initial_epsilon = 1.
final_epsilon = 0.01

In [3]:
def QNetWork(ipt):
    fc1 = fluid.layers.fc(input=ipt, size=24, act='relu')
    fc2 = fluid.layers.fc(input=fc1, size=24, act='relu')
    out = fluid.layers.fc(input=fc2, size=2)
    return out

In [4]:
state = fluid.layers.data(name='state', shape=[4], dtype='float32')
action = fluid.layers.data(name='action', shape=[1], dtype='int64')
reward = fluid.layers.data(name='reward', shape=[], dtype='float32')
next_state = fluid.layers.data(name='next_state', shape=[4], dtype='float32')
done = fluid.layers.data(name='done', shape=[], dtype='bool')

In [5]:
model = QNetWork(state)
print(model.shape)

(-1, 2)


In [6]:
predict_program = fluid.default_main_program().clone()

reward = fluid.layers.clip(reward, min=-1.0, max=1.0)

action_onehot = fluid.layers.one_hot(action, 2)
action_onehot = fluid.layers.cast(action_onehot, dtype='float32')
print(action_onehot)

pred_action_value = fluid.layers.reduce_sum(fluid.layers.elementwise_mul(action_onehot, model), dim=1)


targetQ_predict_value = QNetWork(next_state)
best_v = fluid.layers.reduce_max(targetQ_predict_value, dim=1)
best_v.stop_gradient = True

target = reward + (1.0 - fluid.layers.cast(done, dtype='float32')) * 0.99 * best_v
cost = fluid.layers.square_error_cost(pred_action_value, target)
avg_cost = fluid.layers.reduce_mean(cost)

name: "cast_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 2
    }
    lod_level: 0
  }
}
persistable: false



In [7]:
optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.002)
opt = optimizer.minimize(avg_cost)

In [8]:
place = fluid.CPUPlace()
# place = fluid.CUDAPlace(0)
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

[]

In [9]:
env = gym.make("CartPole-v1")

In [10]:
replay_buffer = deque(maxlen=10000)
for epsilon_id in range(num_episodes):
    state = env.reset()
    for t in range(100):
#         env.render()
        if random.random() < 0.01:
            action = env.action_space.sample()
        else:
            state = np.expand_dims(state, axis=0)
            action = exe.run(predict_program,
                             feed={'state': state.astype('float32')},
                             fetch_list=[model])
            action = np.squeeze(action, axis=0)
            action = np.argmax(action)

#         print(action)
        next_state, reward, done, info = env.step(action)
#         print(next_state, reward, done, info)

        reward = -10 if done else reward
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            break
        
        if len(replay_buffer) >= batch_size:
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = \
                [np.array(a, dtype=np.float32) for a in zip(*random.sample(replay_buffer, batch_size))]
            
            batch_action = np.expand_dims(batch_action, -1)
            batch_next_state = np.expand_dims(batch_next_state, axis=1)
            
            print(batch_state.shape)
            print(batch_action.shape)
            print(batch_reward.shape)
            print(batch_next_state.shape)
            print(batch_done.shape)
            
            out = exe.run(fluid.default_main_program(),
                          feed={'state': batch_state.astype('float32'),
                                'action': batch_action.astype('int64'),
                                'reward': batch_reward,
                                'next_state': batch_next_state.astype('float32'),
                                'done': batch_done},
                         fetch_list=[avg_cost])
            print(out)


(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.8471681], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.8029749], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.7063291], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.6973057], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.70972556], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.72346705], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.8270085], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.8694595], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.76120037], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.69810486], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.8454978], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.56217265], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.6270462

[array([0.18745516], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.1877429], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.3782645], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.16390799], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.19641382], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2268466], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.11932229], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.21744213], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.19731829], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2695811], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.13008769], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2732512], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.24915563], dtype=float32)]
(32, 1, 4)
(32, 1)
(

(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.16504326], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.14532322], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.15605083], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.16934675], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.27984545], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2692145], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.15865578], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.13448389], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.22604412], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.09084245], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.19090588], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.1123603], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.11

[array([0.10281791], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2099959], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.11878694], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.13857847], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.12853384], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.1617262], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.14952046], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.21556687], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.08575621], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.12189537], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.24865413], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.11887767], dtype=float32)]
(32, 1, 4)
(32, 1)
(32,)
(32, 1, 4)
(32,)
[array([0.2472764], dtype=float32)]
(32, 1, 4)
(32, 1)

ValueError: setting an array element with a sequence.